In [1]:
import sys
import time
import os
import subprocess
import math
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, Column 
from scipy.stats import linregress
from scipy import interpolate
from scipy import polyval, polyfit
from scipy.optimize import curve_fit
from scipy import odr
import pylab as py
from matplotlib import gridspec
import sklearn.datasets as ds
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import corner
import emcee
import scipy.optimize as op
from scipy.linalg import cholesky, inv,det
from scipy.optimize import minimize
import random
from astropy.table import Table, Column
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [2]:
#################################################################
def Vh2V3k(el,b, Vh):
  
    alpha = np.pi / 180.
    cosb = np.cos(b*alpha)
    sinb = np.sin(b*alpha)
    cosl = np.cos(el*alpha)
    sinl = np.sin(el*alpha)
    
    v3k = Vh-25.2*cosl*cosb-245.7*sinl*cosb+276.8*sinb

    return v3k

#################################################################

def Vcmb2Vmod(Vcmb, omegam=0.27):
    
    omegal=1.-omegam
    c=299800.
    z=Vcmb/c
    q0=0.5*(omegam-2.*omegal)
    fmod=1.+0.5*(1.-q0)*z-(1./6)*(1.-q0-3.*q0**2+1.)*z**2
    Vmod=c*z*fmod

    return Vmod
#################################################################

def Vh2Vls(el,b, Vh):
  
    alpha = np.pi / 180.
    cosb = np.cos(b*alpha)
    sinb = np.sin(b*alpha)
    cosl = np.cos(el*alpha)
    sinl = np.sin(el*alpha)
    
    vls = Vh-26.*cosl*cosb+317.*sinl*cosb-8.*sinb

    return vls
#################################################################

In [3]:
leda = pd.read_csv('Leda_bar.csv', delimiter='|')
leda = leda.set_index('PGC')

print(len(leda))

133229


/home/ehsan/anaconda3/envs/py2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (69,70,71,72,73,74,75,76) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df = pd.read_csv('VirgoFornax_sbf.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df = df.set_index('PGC')

ln10 = np.log(10.)
df['DM_VirFor']  = df["DM"]
df['eDM_VirFor'] = df["e_dm"]
df = df.rename(columns={"PGC1": "PGC1_VirFor"})

print(len(df))

how = 'inner'
df_VirFor = df.join(leda, lsuffix='_VirFor', rsuffix='_leda', how=how)

print(len(df_VirFor))

df_VirFor["Vcmb_VirFor"] = Vh2V3k(df_VirFor["l2"], df_VirFor["b2"], df_VirFor["v"])

df_VirFor.head()

134
134


,PGC1_VirFor,J2000,Name,g-z,e_gz,m_sbf,e_msbf,DM,e_dm,dist,...,mabs,objname,objtype,type,bar,ring,multiple,compactness,angclass,Vcmb_VirFor
PGC,,,,,,,,,,,,,,,,,,,,,
12636,12651,J032222.7-372351,FCC19,1.066,0.025,29.258,0.036,31.532,0.074,20.2,...,-15.96,ESO301-008,G,S0,NaN,NaN,NaN,NaN,NaN,1317.703371
12651,12651,J032241.7-371230,FCC21,1.368,0.007,29.676,0.020,31.607,0.065,21.0,...,-22.52,NGC1316,G,S0,B,NaN,NaN,NaN,NaN,1680.605238
12691,12651,J032337.3-354642,FCC26,0.830,0.025,28.974,0.055,31.491,0.139,19.9,...,-16.86,ESO357-025,G,E-SO,B,NaN,NaN,NaN,NaN,1631.262951
12825,12923,J032602.2-325340,FCC43,1.154,0.007,29.283,0.039,31.483,0.073,19.8,...,-17.41,IC1919,G,E-SO,NaN,NaN,NaN,NaN,NaN,1214.534307
12848,13418,J032632.2-354249,FCC47,1.298,0.013,29.271,0.040,31.314,0.075,18.3,...,-18.17,NGC1336,G,E-SO,NaN,NaN,NaN,NaN,NaN,1311.507591


In [5]:
df = pd.read_csv('Hydra_Centaurus_sbf.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df = df.set_index('PGC')

ln10 = np.log(10.)
df['DM_HydCen']  = df["DM"]
df['eDM_HydCen'] = df["e_dm"]
df = df.rename(columns={"PGC1": "PGC1_HydCen"})

print(len(df))
how = 'inner'
df_HydCen = df.join(leda, lsuffix='_HydCen', rsuffix='_leda', how=how)
df_HydCen["Vcmb_HydCen"] = Vh2V3k(df_HydCen["l2"], df_HydCen["b2"], df_HydCen["Vh"])
print(len(df_HydCen))

df_HydCen.head()

31
31


,Name,PGC1_HydCen,Nest,GpID,RAJ,DeJ,Vo,V-I,Vh,Type,...,mabs,objname,objtype,type,bar,ring,multiple,compactness,angclass,Vcmb_HydCen
PGC,,,,,,,,,,,,,,,,,,,,,
31438,N3308,31478,100006,487,10 36 22.3,-27 26 17,12.10,1.28,3537,SB0(2)[SAB(s)0-],...,-20.49,NGC3308,G,E-SO,B,NaN,NaN,NaN,NaN,3880.639884
31466,N3311,31478,100006,487,10 36 35.7,-27 31 05,11.90,1.21,4068,E1[E3],...,-21.65,NGC3309,G,E,NaN,NaN,NaN,NaN,NaN,4411.509218
31478,N3309,31478,100006,487,10 36 42.7,-27 31 42,10.90,1.15,3713,S0(2)[E+2],...,-21.27,NGC3311,G,E-SO,NaN,NaN,NaN,NaN,NaN,4056.513803
43058,52,43296,100003,441,12 45 44.3,-41 02 58,17.86,1.09,2609,"dE,N",...,-17.54,PGC043058,G,S?,NaN,NaN,NaN,NaN,NaN,2893.207165
43280,61,43296,100003,441,12 48 39.7,-41 16 05,16.26,1.14,2910,"dE,N",...,-16.20,PGC043280,G,E,NaN,NaN,NaN,NaN,NaN,3192.300680


In [6]:
df = pd.read_csv('Tonry_sbf.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df = df.set_index('PGC')

ln10 = np.log(10.)
df['DM_Tonry']  = df["Modc"]
df['eDM_Tonry'] = df["e_Mod"]
df = df.rename(columns={"PGC1": "PGC1_Tonry"})


print(len(df))
how = 'inner'
df_Tonry = df.join(leda, lsuffix='_Tonry', rsuffix='_leda', how=how)
df_Tonry["Vcmb_Tonry"] = Vh2V3k(df_Tonry["l2"], df_Tonry["b2"], df_Tonry["V_hel"])
print(len(df_Tonry))

df_Tonry.head()

297
297


,objname_Tonry,Modc,PGC1_Tonry,Nest,al2000_Tonry,de2000_Tonry,V_hel,T,GrpID,AgB,...,mabs,objname_leda,objtype,type,bar,ring,multiple,compactness,angclass,Vcmb_Tonry
PGC,,,,,,,,,,,,,,,,,,,,,
218,N7814,30.45,218,211714,0.813,16.146,1059,2,0,0.19,...,-20.15,NGC7814,G,Sab,NaN,NaN,NaN,NaN,NaN,701.544501
1160,N0063,31.30,1160,212242,4.440,11.449,1160,0,0,0.48,...,-19.04,NGC0063,G,E-SO,NaN,NaN,NaN,NaN,NaN,804.882440
2004,N0147,24.24,2557,0,8.298,48.508,-193,-5,282,0.75,...,-14.30,NGC0147,G,E,NaN,NaN,NaN,D,NaN,-455.615596
2329,N0185,23.82,2557,0,9.742,48.338,-202,-5,282,0.79,...,-14.65,NGC0185,Q,E,NaN,NaN,NaN,NaN,S2,-463.091804
2555,N0221,24.35,2557,0,10.675,40.865,-200,-6,282,0.35,...,-15.89,NGC0221,G,E,NaN,NaN,NaN,NaN,NaN,-486.492109


In [7]:
df = pd.read_csv('HST_IR_sbf.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df = df.set_index('PGC')

ln10 = np.log(10.)
df['DM_HSTIR']  = df["DM"]
df['eDM_HSTIR'] = df["edm"]
df_HSTIR = df.rename(columns={"PGC1": "PGC1_HSTIR"})

df_HSTIR["Vcmb_HSTIR"] = Vh2V3k(df_HSTIR["glon"], df_HSTIR["glat"], df_HSTIR["Vhel"])

df_HSTIR.head()

,Name,PGC1_HSTIR,Nest,DM,edm,Vhel,Ty,RA_DEJ,glon,glat,sgl,sgb,Ag,DM_HSTIR,eDM_HSTIR,Vcmb_HSTIR
PGC,,,,,,,,,,,,,,,,
1037,NGC0057,1037,200319,34.126,0.074,5449,-4.9,J001530.9+171942,110.8305,-44.6976,310.9031,13.7833,0.335,34.126,0.074,5097.443600
3455,NGC0315,3455,200070,34.167,0.068,4974,-4.1,J005748.9+302108,124.5630,-32.4992,326.0060,7.2773,0.280,34.167,0.068,4666.687342
3982,NGC0383,4224,200005,34.100,0.076,5057,-2.9,J010724.9+322446,126.8388,-30.3378,328.5287,5.8072,0.307,34.100,0.076,4760.516577
4224,NGC0410,4224,200005,33.937,0.070,5276,-4.3,J011058.9+330908,127.6335,-29.5410,329.4427,5.2765,0.254,33.937,0.070,4983.628463
5037,NGC0495,5098,200006,34.049,0.059,4104,0.2,J012256.2+332813,130.4359,-28.9420,330.4440,2.9656,0.311,34.049,0.059,3820.698741


In [8]:
df = pd.read_csv('Cantiello18_sbf.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df = df.set_index('PGC')

ln10 = np.log(10.)
df['DM_Cantiello18']  = df["DM"]
df['eDM_Cantiello18'] = df["edm"]
df = df.rename(columns={"PGC1": "PGC1_Cantiello18"})

print(len(df))
how = 'inner'
df_Cantiello18 = df.join(leda, lsuffix='_Cantiello18', rsuffix='_leda', how=how)
df_Cantiello18["Vcmb_Cantiello18"] = Vh2V3k(df_Cantiello18["l2"], df_Cantiello18["b2"], df_Cantiello18["Vhel"])
print(len(df_Cantiello18))

df_Cantiello18.head()

89
89


,Name,VCC,dist,eD,DM,edm,Vhel,PGC1_Cantiello18,Nest,Gp,...,mabs,objname,objtype,type,bar,ring,multiple,compactness,angclass,Vcmb_Cantiello18
PGC,,,,,,,,,,,,,,,,,,,,,
38890,NGC4168,VCC0049,31.4,1.9,32.49,0.13,2273,38890,130002,M,...,-20.78,NGC4168,G,E,NaN,NaN,NaN,NaN,S1.9,2608.863643
39246,NGC4216,VCC0167,18.1,1.9,31.29,0.23,134,41220,100002,V,...,-20.90,NGC4216,G,SABb,B,NaN,NaN,NaN,NaN,468.471293
39328,NGC4224,VCC0199,21.5,1.3,31.66,0.13,2587,39659,120002,W/W'?,...,-20.52,NGC4224,G,Sa,NaN,NaN,NaN,NaN,NaN,2931.871275
39384,NGC4233,VCC0220,31.1,2.2,32.46,0.15,2275,39659,120002,W,...,-19.81,NGC4233,G,S0,B,NaN,NaN,NaN,NaN,2619.362026
39389,NGC4235,VCC0222,26.6,2.4,32.12,0.19,2423,39659,120002,W,...,-20.82,NGC4235,G,Sa,NaN,NaN,NaN,NaN,S1.2,2768.027945


In [9]:
how = 'outer'
df = df_HSTIR.join(df_Tonry, lsuffix='_l0', rsuffix='_', how=how)
df = df.join(df_Cantiello18, lsuffix='_l1', rsuffix='_', how=how)
df = df.join(df_VirFor, lsuffix='_l2', rsuffix='_', how=how)
df = df.join(df_HydCen, lsuffix='_l3', rsuffix='_', how=how)
     

catalogs = ["VirFor", "Tonry","Cantiello18"] #,"HydCen"]"HSTIR"

ss = []
for cat in catalogs:
    ss += ['PGC1_'+cat, "DM_"+cat, "eDM_"+cat, "Vcmb_"+cat]

df = df[ss]

ss = []
for cat in catalogs:
    ss += ['PGC1_'+cat]   
df['pgc1'] = df[ss].median(axis=1)
df = df[~df['pgc1'].isna()]


print(len(df))

df.head()

425


/home/ehsan/anaconda3/envs/py2/lib/python2.7/site-packages/numpy/lib/function_base.py:3405: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


,PGC1_VirFor,DM_VirFor,eDM_VirFor,Vcmb_VirFor,PGC1_Tonry,DM_Tonry,eDM_Tonry,Vcmb_Tonry,PGC1_Cantiello18,DM_Cantiello18,eDM_Cantiello18,Vcmb_Cantiello18,pgc1
PGC,,,,,,,,,,,,,
218,NaN,NaN,NaN,NaN,218.0,30.45,0.14,701.544501,NaN,NaN,NaN,NaN,218.0
1160,NaN,NaN,NaN,NaN,1160.0,31.30,0.33,804.882440,NaN,NaN,NaN,NaN,1160.0
2004,NaN,NaN,NaN,NaN,2557.0,24.24,0.16,-455.615596,NaN,NaN,NaN,NaN,2557.0
2329,NaN,NaN,NaN,NaN,2557.0,23.82,0.16,-463.091804,NaN,NaN,NaN,NaN,2557.0
2555,NaN,NaN,NaN,NaN,2557.0,24.35,0.08,-486.492109,NaN,NaN,NaN,NaN,2557.0


In [10]:
for cat in catalogs:
    df['Vmod_'+cat] = Vcmb2Vmod(df['Vcmb_'+cat])
    logD = (df['DM_'+cat]-25)/5.
    df['logH_'+cat] = np.log10(df['Vmod_'+cat]) - logD 
    
    print cat, 10**df['logH_'+cat][df['Vmod_'+cat]>4000].median()

VirFor 4913.205760729554
Tonry 117.31859911875317
Cantiello18 nan


/media/Data/Home/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log10
  after removing the cwd from sys.path.


In [11]:
df["PGC1_cf4"] = df["PGC1_VirFor"]

ind = (df["PGC1_cf4"].isna()) | (df["PGC1_cf4"]==0)
print len(df["PGC1_cf4"][ind])

for cat in catalogs:
    ind = (df["PGC1_cf4"].isna()) | (df["PGC1_cf4"]==0)
    df["PGC1_cf4"][ind] = df["PGC1_"+cat][ind]
    

ind = (df["PGC1_cf4"].isna()) | (df["PGC1_cf4"]==0)
print len(df["PGC1_cf4"][ind])

ind = (df["PGC1_cf4"].isna()) | (df["PGC1_cf4"]==0)
df["PGC1_cf4"][ind] = np.nan

291
0


In [12]:
df["Vmod_cf4"] = df["Vmod_VirFor"]

ind = (df["Vmod_cf4"].isna()) | (df["Vmod_cf4"]==0)
print len(df["Vmod_cf4"][ind])

for cat in catalogs:
    ind = (df["Vmod_cf4"].isna()) | (df["Vmod_cf4"]==0)
    df["Vmod_cf4"][ind] = df["Vmod_"+cat][ind]
    
ind = (df["Vmod_cf4"].isna()) | (df["Vmod_cf4"]==0)
df["Vmod_cf4"][ind] = np.nan

292


In [13]:
df["Vcmb_cf4"] = df["Vcmb_VirFor"]

ind = (df["Vcmb_cf4"].isna()) | (df["Vcmb_cf4"]==0)
print len(df["Vcmb_cf4"][ind])

for cat in catalogs:
    ind = (df["Vcmb_cf4"].isna()) | (df["Vcmb_cf4"]==0)
    df["Vcmb_cf4"][ind] = df["Vcmb_"+cat][ind]
    
ind = (df["Vcmb_cf4"].isna()) | (df["Vcmb_cf4"]==0)
df["Vcmb_cf4"][ind] = np.nan

292


In [14]:
df.head()

,PGC1_VirFor,DM_VirFor,eDM_VirFor,Vcmb_VirFor,PGC1_Tonry,DM_Tonry,eDM_Tonry,Vcmb_Tonry,PGC1_Cantiello18,DM_Cantiello18,...,pgc1,Vmod_VirFor,logH_VirFor,Vmod_Tonry,logH_Tonry,Vmod_Cantiello18,logH_Cantiello18,PGC1_cf4,Vmod_cf4,Vcmb_cf4
PGC,,,,,,,,,,,,,,,,,,,,,
218,NaN,NaN,NaN,NaN,218.0,30.45,0.14,701.544501,NaN,NaN,...,218.0,NaN,NaN,702.852731,1.756864,NaN,NaN,218.0,702.852731,701.544501
1160,NaN,NaN,NaN,NaN,1160.0,31.30,0.33,804.882440,NaN,NaN,...,1160.0,NaN,NaN,806.604270,1.646661,NaN,NaN,1160.0,806.604270,804.882440
2004,NaN,NaN,NaN,NaN,2557.0,24.24,0.16,-455.615596,NaN,NaN,...,2557.0,NaN,NaN,-455.063128,NaN,NaN,NaN,2557.0,-455.063128,-455.615596
2329,NaN,NaN,NaN,NaN,2557.0,23.82,0.16,-463.091804,NaN,NaN,...,2557.0,NaN,NaN,-462.521051,NaN,NaN,NaN,2557.0,-462.521051,-463.091804
2555,NaN,NaN,NaN,NaN,2557.0,24.35,0.08,-486.492109,NaN,NaN,...,2557.0,NaN,NaN,-485.862203,NaN,NaN,NaN,2557.0,-485.862203,-486.492109


In [15]:
myDict = {}

myDict["VirFor"] = 0.
myDict["Tonry"] = 0.018
myDict["Cantiello18"] = 0.005

for cat in catalogs:
    df['DM_'+cat] += myDict[cat]
    df['w_'+cat] = (1.-0)/df['eDM_'+cat]**2
    df['xw_'+cat] = (df['DM_'+cat])*df['w_'+cat]

sw = ['w_'+cat for cat in catalogs]
sx = ['xw_'+cat for cat in catalogs]

Err2 = 1./df[sw].sum(axis=1)

df["eDM_av"] = np.sqrt(Err2)
df["DM_av"] = df[sx].sum(axis=1)*Err2   

In [16]:
df['av_flag'] = 0*df['DM_av']
df['av_flag'][df['DM_av'].isna()] = 1

In [17]:
for cat in catalogs:
    df['w_'+cat] = 1./df['eDM_'+cat]**2
    df['xw_'+cat] = df['DM_'+cat]*df['w_'+cat]

Err2 = 1./df[sw].sum(axis=1)
df["DM_av_"] = df[sx].sum(axis=1)*Err2
df["eDM_av_"] = np.sqrt(Err2)

ind = df["av_flag"] == 1
df["eDM_av"][ind] = df["eDM_av_"][ind]
df["DM_av"][ind] = df["DM_av_"][ind]

In [18]:
catalogs

['VirFor', 'Tonry', 'Cantiello18']

In [19]:
df.columns

Index([u'PGC1_VirFor', u'DM_VirFor', u'eDM_VirFor', u'Vcmb_VirFor',
       u'PGC1_Tonry', u'DM_Tonry', u'eDM_Tonry', u'Vcmb_Tonry',
       u'PGC1_Cantiello18', u'DM_Cantiello18', u'eDM_Cantiello18',
       u'Vcmb_Cantiello18', u'pgc1', u'Vmod_VirFor', u'logH_VirFor',
       u'Vmod_Tonry', u'logH_Tonry', u'Vmod_Cantiello18', u'logH_Cantiello18',
       u'PGC1_cf4', u'Vmod_cf4', u'Vcmb_cf4', u'w_VirFor', u'xw_VirFor',
       u'w_Tonry', u'xw_Tonry', u'w_Cantiello18', u'xw_Cantiello18', u'eDM_av',
       u'DM_av', u'av_flag', u'DM_av_', u'eDM_av_'],
      dtype='object')

In [20]:
ss = ['PGC1_cf4', 'Vcmb_cf4', 'Vmod_cf4', "DM_av", "eDM_av"]
for cat in catalogs:
    ss += ["DM_"+cat, "eDM_"+cat]

df = df.fillna(0)

df = df[ss]


df = df.rename(columns={'PGC1_cf4':'PGC1',
                        'Nest_cf4':'Nest',
                        'Vcmb_cf4': 'Vcmb',
                        'Vmod_cf4': 'Vmod'
                       })

In [21]:
len(df)

425

In [22]:
df = df[df["DM_av"]!=0]

In [23]:
len(df)

425

In [24]:
df.to_csv("SBF_CF4_components_bar.csv", sep='|')

In [25]:
table   = np.genfromtxt('SBF_CF4_components_bar.csv' , delimiter='|', 
                        filling_values=-1, names=True, dtype=None, encoding=None)

colnames = table.dtype.names

## table is a structured array
myTable = {}
for name in table.dtype.names:
    myTable[name] = table[name]
table = myTable
## table is now a dictionary

myTable = Table()

for key in colnames:
    if key in ["PGC", "PGC1"]:
        myTable.add_column(Column(data=table[key], name=key, dtype=np.dtype(int))) 
    elif key in ["Vcmb", "Vmod"]:
        myTable.add_column(Column(data=table[key], name=key, dtype=np.dtype(int)))
    else:
        myTable.add_column(Column(data=table[key], name=key, format='%0.2f'))

## to be used on EDD
myTable.write('SBF_CF4_components_bar', format='ascii.fixed_width',delimiter='|', bookend=False, overwrite=True)